In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import os
import csv
from pandas.api.types import CategoricalDtype
low_memory = False

In [2]:
#make the path, read in the file, and filter so that only CA stats remain
ca_mortalities_path = './Resources/CA_covid_deaths_by_county/covid_deaths_usafacts.csv'
ca_mortalities = pd.read_csv(ca_mortalities_path, encoding='utf-8')
ca_mortalities = pd.DataFrame(ca_mortalities)
ca_mortalities = ca_mortalities.loc[ca_mortalities['State'] == 'CA']
ca_mortalities.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2021-07-30,2021-07-31,2021-08-01,2021-08-02,2021-08-03,2021-08-04,2021-08-05,2021-08-06,2021-08-07,2021-08-08
190,0,Statewide Unallocated,CA,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
191,6001,Alameda County,CA,6,0,0,0,0,0,0,...,1265,1265,1265,1270,1270,1270,1274,1275,1275,1275
192,6003,Alpine County,CA,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
193,6005,Amador County,CA,6,0,0,0,0,0,0,...,48,48,48,48,48,48,48,48,48,48
194,6007,Butte County,CA,6,0,0,0,0,0,0,...,204,204,204,204,204,204,204,204,204,204


In [3]:
#filter out only the last days of the month to get the monthly aggregates
ca_deaths_by_month = ca_mortalities[['County Name','2020-08-31','2020-09-30','2020-10-31',
                                    '2020-11-30', '2020-12-31', '2021-01-31','2021-02-28',
                                    '2021-03-31','2021-04-30','2021-05-31','2021-06-30',
                                    '2021-07-31']]
#ca_deaths_by_month.set_index('County Name')

ca_deaths_by_month.head()

,County Name,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31
190,Statewide Unallocated,0,0,0,0,0,0,0,0,0,0,0,0
191,Alameda County,257,412,463,512,625,974,1241,1212,1263,1253,1257,1265
192,Alpine County,0,0,0,0,0,0,0,0,0,0,0,0
193,Amador County,14,16,16,16,24,33,41,45,46,46,46,48
194,Butte County,23,44,53,60,96,138,160,191,196,189,200,204


In [4]:
#rename columns so that the date is now the month name
renamed_ca_deaths = ca_deaths_by_month.rename(columns={'2020-08-31':'August','2020-09-30':'September',
                    '2020-10-31':'October','2020-11-30':'November','2020-12-31':'December',
                    '2021-01-31':'January','2021-02-28':'February','2021-03-31':'March',
                    '2021-04-30':'April', '2021-05-31':'May','2021-06-30':'June','2021-07-31':'July'})

renamed_ca_deaths.head()

,County Name,August,September,October,November,December,January,February,March,April,May,June,July
190,Statewide Unallocated,0,0,0,0,0,0,0,0,0,0,0,0
191,Alameda County,257,412,463,512,625,974,1241,1212,1263,1253,1257,1265
192,Alpine County,0,0,0,0,0,0,0,0,0,0,0,0
193,Amador County,14,16,16,16,24,33,41,45,46,46,46,48
194,Butte County,23,44,53,60,96,138,160,191,196,189,200,204


In [5]:
#renamed_ca_deaths = renamed_ca_deaths.transpose()
#even though transposed, searching for 'County Name' will still give the name of
#all counties
#renamed_ca_deaths['County Name'] 
#renamed_ca_deaths.reset_index()
renamed_ca_deaths

,County Name,August,September,October,November,December,January,February,March,April,May,June,July
190,Statewide Unallocated,0,0,0,0,0,0,0,0,0,0,0,0
191,Alameda County,257,412,463,512,625,974,1241,1212,1263,1253,1257,1265
192,Alpine County,0,0,0,0,0,0,0,0,0,0,0,0
193,Amador County,14,16,16,16,24,33,41,45,46,46,46,48
194,Butte County,23,44,53,60,96,138,160,191,196,189,200,204
195,Calaveras County,2,14,21,22,22,23,47,50,50,55,55,57
196,Colusa County,6,6,6,6,8,10,13,14,14,14,15,16
197,Contra Costa County,175,209,246,261,331,529,681,767,795,805,816,831
198,Del Norte County,1,1,1,1,2,3,5,6,6,7,7,7
199,El Dorado County,2,4,4,4,22,79,100,105,107,110,113,116


In [6]:

renamed_ca_deaths.to_csv('./Resources/CA_covid_deaths_by_county/ca_covid_deaths_by_county.csv',index=False)